Credit Risks
Step 1: Download the dataset from the following link:  https://www.openml.org/d/31

Step 2: Read the dataset into a pandas dataframe.

In [3]:
import pandas as pd
from scipy.io import arff

data, meta = arff.loadarff("dataset_31_credit-g.arff")
df = pd.DataFrame(data)
print(df.head(6))


  checking_status  duration                     credit_history  \
0           b'<0'       6.0  b'critical/other existing credit'   
1     b'0<=X<200'      48.0                   b'existing paid'   
2  b'no checking'      12.0  b'critical/other existing credit'   
3           b'<0'      42.0                   b'existing paid'   
4           b'<0'      24.0              b'delayed previously'   
5  b'no checking'      36.0                   b'existing paid'   

                  purpose  credit_amount       savings_status employment  \
0             b'radio/tv'         1169.0  b'no known savings'     b'>=7'   
1             b'radio/tv'         5951.0              b'<100'  b'1<=X<4'   
2            b'education'         2096.0              b'<100'  b'4<=X<7'   
3  b'furniture/equipment'         7882.0              b'<100'  b'4<=X<7'   
4              b'new car'         4870.0              b'<100'  b'1<=X<4'   
5            b'education'         9055.0  b'no known savings'  b'1<=X<4'   

   i